In [ ]:
# default_exp model.dummy

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# export
import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
import albumentations as A
import albumentations.pytorch.transforms as AT

from pytorch_lightning.metrics.functional import accuracy
from fractal_augmentation.data.cifar import (
    CIFAR10DataModule,
    CIFAR10,
    CIFAR10MandelbrotDataModule,
)
from fractal_augmentation.transforms.fractal import MandelBrotFractalTransform
from loguru import logger

In [ ]:
# export
class LitModel(pl.LightningModule):
    def __init__(
        self, channels, width, height, num_classes, hidden_size=64, learning_rate=2e-4
    ):

        super().__init__()

        self.save_hyperparameters()

        # We take in input dimensions as parameters and use those to dynamically build model.
        self.channels = channels
        self.width = width
        self.height = height
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate

        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(channels * width * height, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, num_classes),
        )

    def forward(self, x):
        x = self.model(x)
        return F.log_softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return {"loss": loss}

    def training_epoch_end(self, outputs):
        loss = torch.mean(torch.tensor([x["loss"] for x in outputs]))

        self.log("train_loss", loss)

    def validation_step(self, batch, batch_idx):

        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        return {"val_loss": loss, "val_acc": acc}

    def validation_epoch_end(self, outputs):

        avg_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()

        self.log_dict({"val_loss": avg_loss, "val_acc": avg_acc})

        logger.info(f"val_loss: {avg_loss}, val_acc: {avg_acc}")

    def test_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)

        acc = accuracy(preds, y)

        return {"test_acc": acc}

    def test_epoch_end(self, outputs):
        avg_acc = torch.stack([x["test_acc"] for x in outputs]).mean()

        self.log("test_acc", avg_acc)

        logger.info(f"test_acc: {avg_acc}")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
dm = CIFAR10DataModule(
    transform=A.Compose(
        [
            A.Resize(32, 32),
            A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std),
            AT.ToTensor(),
        ]
    )
)
model = LitModel(*dm.size(), dm.num_classes, hidden_size=32)
trainer = pl.Trainer(max_epochs=1, progress_bar_refresh_rate=20)
trainer.fit(model, dm)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified



  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 99.7 K
-------------------------------------
99.7 K    Trainable params
0         Non-trainable params
99.7 K    Total params


2020-12-15 19:41:24.237 | INFO     | __main__:validation_epoch_end:62 - val_loss: 2.2639222145080566, val_acc: 0.1875


2020-12-15 19:41:34.929 | INFO     | __main__:validation_epoch_end:62 - val_loss: 1.6878979206085205, val_acc: 0.41600319743156433


1

In [ ]:
trainer.test(model, datamodule=dm)

Files already downloaded and verified


2020-12-15 19:41:38.561 | INFO     | __main__:test_epoch_end:77 - test_acc: 0.41433706879615784



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.4143),
 'val_acc': tensor(0.4160),
 'val_loss': tensor(1.6879)}
--------------------------------------------------------------------------------


[{'val_loss': 1.6878979206085205,
  'val_acc': 0.41600319743156433,
  'test_acc': 0.41433706879615784}]

In [ ]:
dm = CIFAR10DataModule(
    transform=A.Compose(
        [
            A.PadIfNeeded(min_height=36, min_width=36),
            A.RandomCrop(height=32, width=32),
            A.HorizontalFlip(),
            A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std),
            A.Cutout(num_holes=4),
            AT.ToTensor(),
        ]
    )
)
model = LitModel(*dm.size(), dm.num_classes, hidden_size=32)
trainer = pl.Trainer(max_epochs=1, progress_bar_refresh_rate=20)
trainer.fit(model, dm)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified



  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 99.7 K
-------------------------------------
99.7 K    Trainable params
0         Non-trainable params
99.7 K    Total params


2020-12-15 19:41:42.591 | INFO     | __main__:validation_epoch_end:62 - val_loss: 2.3110501766204834, val_acc: 0.09375


2020-12-15 19:41:58.647 | INFO     | __main__:validation_epoch_end:62 - val_loss: 1.7743120193481445, val_acc: 0.36922770738601685


1

In [ ]:
trainer.test(model, datamodule=dm)

Files already downloaded and verified


2020-12-15 19:42:01.947 | INFO     | __main__:test_epoch_end:77 - test_acc: 0.4024560749530792



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.4025),
 'val_acc': tensor(0.3692),
 'val_loss': tensor(1.7743)}
--------------------------------------------------------------------------------


[{'val_loss': 1.7743120193481445,
  'val_acc': 0.36922770738601685,
  'test_acc': 0.4024560749530792}]

In [ ]:
dm = CIFAR10DataModule(
    transform=A.Compose(
        [
            MandelBrotFractalTransform(n_iter=2, deg=2, delta=0.1),
            A.Resize(64, 64),
            A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std),
            AT.ToTensor(),
        ]
    )
)
model = LitModel(3, 64, 64, dm.num_classes, hidden_size=32)
trainer = pl.Trainer(max_epochs=1, progress_bar_refresh_rate=20)
trainer.fit(model, dm)

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified



  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 394 K 
-------------------------------------
394 K     Trainable params
0         Non-trainable params
394 K     Total params
INFO:lightning:
  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 394 K 
-------------------------------------
394 K     Trainable params
0         Non-trainable params
394 K     Total params


2020-12-14 19:13:49.541 | INFO     | __main__:validation_epoch_end:63 - val_loss: 2.3048267364501953, val_acc: 0.15625


2020-12-14 19:18:44.080 | INFO     | __main__:validation_epoch_end:63 - val_loss: 2.0835177898406982, val_acc: 0.21218152344226837


1

In [ ]:
trainer.test(model, datamodule=dm)

Files already downloaded and verified


2020-12-14 19:12:46.892 | INFO     | __main__:test_epoch_end:78 - test_acc: 0.18130990862846375



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.1813),
 'val_acc': tensor(0.3055),
 'val_loss': tensor(1.9136)}
--------------------------------------------------------------------------------


[{'val_loss': 1.9136290550231934,
  'val_acc': 0.30553343892097473,
  'test_acc': 0.18130990862846375}]

In [ ]:
dm = CIFAR10MandelbrotDataModule(
    data_dir="../dev_nbs/cifar10_mandelbrot",
    transform=A.Compose(
        [
            A.Resize(96, 96),
            A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std),
            AT.ToTensor(),
        ]
    ),
    test_transform=A.Compose(
        [
            A.Resize(96, 96),
            A.Normalize(mean=CIFAR10.mean, std=CIFAR10.std),
            AT.ToTensor(),
        ]
    ),
)
model = LitModel(3, 96, 96, dm.num_classes, hidden_size=64)
trainer = pl.Trainer(max_epochs=1, progress_bar_refresh_rate=20)
trainer.fit(model, dm)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 1.8 M 
-------------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params


2020-12-15 19:45:54.493 | INFO     | __main__:validation_epoch_end:62 - val_loss: 2.345595359802246, val_acc: 0.046875


2020-12-15 19:46:30.531 | INFO     | __main__:validation_epoch_end:62 - val_loss: 1.948337197303772, val_acc: 0.2617436349391937


1

In [ ]:
trainer.test(model, datamodule=dm)

2020-12-15 19:46:38.052 | INFO     | __main__:test_epoch_end:77 - test_acc: 0.2696685194969177



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.2697),
 'val_acc': tensor(0.2617),
 'val_loss': tensor(1.9483)}
--------------------------------------------------------------------------------


[{'val_loss': 1.948337197303772,
  'val_acc': 0.2617436349391937,
  'test_acc': 0.2696685194969177}]

In [ ]:
from nbdev.export import notebook2script

notebook2script()

Converted 01_data.cifar.ipynb.
Converted 02_transforms.fractal.ipynb.
Converted 03_model.dummy.ipynb.
Converted 03a_model.resnet18.ipynb.
Converted dev_setup.ipynb.
Converted index.ipynb.
